In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import os
NUM_THREADS = "1"
os.environ["OMP_NUM_THREADS"] = NUM_THREADS
os.environ["OPENBLAS_NUM_THREADS"] = NUM_THREADS
os.environ["MKL_NUM_THREADS"] = NUM_THREADS
os.environ["VECLIB_MAXIMUM_THREADS"] = NUM_THREADS
os.environ["NUMEXPR_NUM_THREADS"] = NUM_THREADS
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np
import warnings; warnings.simplefilter('ignore')

data = pd.read_csv('Country-data.csv')

print(f"Перевірка загальної інформації про дані:\n")
print(data.info())
print("")
print(f"Перевірка на пропущені значення:\n")
print(data.isnull().sum())

# очищення даних 
numeric_data = data.select_dtypes(include=['float64', 'int64'])

# однофакторний аналіз
print(f"Однофакторний аналіз:\n")
for column in data.columns[1:]: 
    plt.figure(figsize=(8, 6))
    sns.histplot(data[column], bins=20, kde=True)
    plt.title(f'{column} distribution')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

# двофакторний аналіз
print(f"Двофакторний аналіз:\n")
plt.figure(figsize=(8, 6))
sns.scatterplot(x='gdpp', y='child_mort', data=data)
plt.title('GDP per Capita vs Child Mortality Rate')
plt.xlabel('GDP per Capita')
plt.ylabel('Child Mortality Rate')
plt.show()

# Кореляційна матриця
print(f"Кореляційна матриця:\n")
plt.figure(figsize=(10, 8))
sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# аналіз викидів
import seaborn as sns
import matplotlib.pyplot as plt

variables = ['gdpp', 'child_mort', 'income']

plt.figure(figsize=(10, 6))
for i, variable in enumerate(variables, 1):
    plt.subplot(1, 3, i)
    sns.boxplot(y=data[variable])
    plt.title(f'Boxplot for {variable}')
    plt.tight_layout()

plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

pd.set_option('display.max_rows', None)
data = pd.read_csv('Country-data.csv')
features = ['gdpp', 'child_mort', 'income']

optimal_k = 3 
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
data['Cluster'] = kmeans.fit_predict(data[features])

for cluster_index in range(optimal_k):
    cluster_countries = data[data['Cluster'] == cluster_index]['country']
    print(f"Країни у кластері {cluster_index}:")
    print(cluster_countries)

# візуалізація кластерів
for i, col1 in enumerate(features):
    for j, col2 in enumerate(features):
        if i < j:
            plt.figure(figsize=(8, 6))
            for cluster in range(optimal_k):
                cluster_data = data[data['Cluster'] == cluster]
                plt.scatter(cluster_data[col1], cluster_data[col2], label=f'Cluster {cluster}')
            plt.xlabel(col1)
            plt.ylabel(col2)
            plt.title(f'{col1} vs {col2}')
            plt.legend()
            plt.show()

# аналіз кластерів
cluster_means = data.groupby('Cluster')[features].mean()
print("Середні значення для кожного кластера:")
print(cluster_means)

# кластери, що потребують допомоги
# розглядаємо кластери з низьким GDPP(ввп на душу населення),високим child_mort(смертністю дітей) та низьким income як ты, що потребую допомоги:

needy_clusters = cluster_means[(cluster_means['gdpp'] < cluster_means['gdpp'].median()) &
                               (cluster_means['child_mort'] > cluster_means['child_mort'].median()) &
                               (cluster_means['income'] < cluster_means['income'].median())]
print("\nКластери, які гостро потребують допомоги:")
print(needy_clusters)

needy_cluster_indices = data[data['Cluster'].isin(needy_clusters.index)]
needy_countries = needy_cluster_indices['country']
print("\nКраїни, які гостро потребують допомоги:")
print(needy_countries)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from kneed import KneeLocator
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

data = pd.read_csv('Country-data.csv')
features = data.drop('country', axis=1)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# визначення оптимального k за допомогою kneed
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)

kl = KneeLocator(range(1, 11), sse, curve="convex", direction="decreasing")
optimal_k = kl.elbow

plt.figure(figsize=(8, 6))
plt.plot(range(1, 11), sse, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Sum of squared distances')
plt.title('Elbow Method for Optimal k')
plt.vlines(optimal_k, plt.ylim()[0], plt.ylim()[1], linestyles='dashed', colors='red', label=f'Optimal k: {optimal_k}')
plt.legend()
plt.show()

print(f'Optimal k is: {optimal_k}')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

pd.set_option('display.max_rows', None)
data = pd.read_csv('Country-data.csv')
features = ['gdpp', 'child_mort', 'income']

optimal_k = 4  
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
data['Cluster'] = kmeans.fit_predict(data[features])

# візуалізація кластерів
for i, col1 in enumerate(features):
    for j, col2 in enumerate(features):
        if i < j:
            plt.figure(figsize=(8, 6))
            for cluster in range(optimal_k):
                cluster_data = data[data['Cluster'] == cluster]
                plt.scatter(cluster_data[col1], cluster_data[col2], label=f'Cluster {cluster}')
            plt.xlabel(col1)
            plt.ylabel(col2)
            plt.title(f'{col1} vs {col2}')
            plt.legend()
            plt.show()

# аналіз кластерів
cluster_means = data.groupby('Cluster')[features].mean()
print("Середні значення для кожного кластера:")
print(cluster_means)

# розглядаємо кластери з низьким GDPP, високим child_mort та низьким income як ті, що потребують допомоги:
needy_clusters = cluster_means[(cluster_means['gdpp'] < cluster_means['gdpp'].median()) &
                               (cluster_means['child_mort'] > cluster_means['child_mort'].median()) &
                               (cluster_means['income'] < cluster_means['income'].median())]
print("\nКластери, які гостро потребують допомоги:")
print(needy_clusters)

# Вивід країн, які гостро потребують допомоги для кожного кластера
for cluster_index in needy_clusters.index:
    cluster_countries = data[data['Cluster'] == cluster_index]['country']
    print(f"\nКраїни у кластері {cluster_index}, які гостро потребують допомоги:")
    print(cluster_countries)

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans

data = pd.read_csv('Country-data.csv')
features = ['gdpp', 'child_mort', 'income']

optimal_k = 4 
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
data['Cluster'] = kmeans.fit_predict(data[features])

cluster_means = data.groupby('Cluster')[features].mean()

needy_clusters = cluster_means[(cluster_means['gdpp'] < cluster_means['gdpp'].median()) &
                               (cluster_means['child_mort'] > cluster_means['child_mort'].median()) &
                               (cluster_means['income'] < cluster_means['income'].median())]

# обираємо країни, які гостро потребують допомоги для кожного кластера
needy_countries = []
for cluster_index in needy_clusters.index:
    cluster_countries = data[data['Cluster'] == cluster_index]['country']
    needy_countries.extend(cluster_countries)

# 10 країн, які найбільше потребують допомоги
top_needy_countries = data[data['country'].isin(needy_countries)]
top_needy_countries = top_needy_countries.sort_values(by=features, ascending=[True, False, True]).head(10)

print("Топ 10 країн, які найбільше потребують допомоги:")
print(top_needy_countries[['country', 'gdpp', 'child_mort', 'income']])